In [1]:

import _init_
from constants import *
from set_path import *
from config import *
from functions import *

In [2]:
file = '/g/data/jh2/av5889/quokka_myrepo/quokka/extern/grackle_data_files/input/CloudyData_UVB=HM2012.h5'
grackle = h5py.File(file)
array = grackle['CoolingRates/Primordial/MMW'][()]
#density(1.e-6, 1.e4), redshift(0,15), temperature(10., 1.e9)
table = array[:,0,:]
table_nH   = np.logspace(-6, 4, array.shape[0])
table_temp = np.logspace(1,  9, array.shape[2])


In [3]:

i=0
bins = 100
egas_arr = np.logspace(-16., -5., bins)
nH_arr   = np.logspace(-6.0, 4.0, int(bins))
T = np.zeros((egas_arr.shape[0],nH_arr.shape[0]))

for egas in egas_arr:
    j=0
    for nH in nH_arr:
        C = (gamma - 1.) * egas / (boltzmann_constant_cgs*nH)
        minT = C*np.amin(table)
        maxT = C*np.amax(table)
        def func(T):
            mu = interpolate.interp2d(table_temp, table_nH, table,\
                              kind='linear', copy=True, bounds_error=False, fill_value=None)
            return C*mu(T,nH)[0] - T

        T[i,j] = scipy.optimize.toms748(func, minT, maxT)
        j+=1
    i+=1

    
# temperature_table = interpolate.RectBivariateSpline(egas_arr, nH_arr, T)


In [4]:
data_path = os.path.join(scratch, 'sims/DiodeBC/GPU/')
output_folder = os.path.join(fig_path, 'DiodeBC/GPU/Slice/')

In [5]:
def makeSlicePlot(queue):
    while True:
        item = queue.get()
        if item is None:
            break
        
        f = item[0]
        infile = item[1]
        
        infile   = os.path.join(data_path, 'metal_uniform.in')
        dom_min, dom_max, ncells = getdomain(infile)
        fac = 1
        zrange = np.linspace(dom_min[2], dom_max[2], (fac*int(ncells[2])))
        xrange = np.linspace(dom_min[0], dom_max[0], (fac*int(ncells[0])))
        yrange = np.linspace(dom_min[1], dom_max[1], (fac*int(ncells[1])))
        
        
        
        Lx = (dom_max[0]- dom_min[0])
        Ly = (dom_max[1]- dom_min[1])
        Lz = (dom_max[2]- dom_min[2])
        
        plane = int(ncells[1]*fac/2)
        
    
        inputfile = os.path.join(data_path, f)
        ds   = yt.load(inputfile)
        timestep = ds.current_time.to('Myr')
        data = ds.covering_grid(level=lev, left_edge=dom_min, dims=ds.domain_dimensions * fac)
        
        
        lev = 0

        rho_gas = np.array(data['gasDensity'])
        egas    = np.array(data['gasEnergy'])
        eint    = np.array(data['gasInternalEnergy'])
        vz = np.array(data['z-GasMomentum'])/rho_gas
        vx = np.array(data['x-GasMomentum'])/rho_gas
        vy = np.array(data['y-GasMomentum'])/rho_gas

        
        fig, ax = plt.subplots(1, 2, gridspec_kw = {'wspace':0.02, 'hspace':0.02},figsize=(8, 32))
    
        cbarx = 0.13
        cbheight = 0.04
        cbary = 0.89
        cblen = 0.3
        dx1 = 0.13
        cbtitlex = 0.1
        cbtitley = 16.5
        plane = (int)(ncells[1]/2)
       

        plot = ax[0].pcolormesh(yrange/kpc,zrange/kpc, np.transpose(rho_gas[:,plane,:]/mp),\
                            norm=mcolors.LogNorm(vmin=1.e-6, vmax=4.e2),
                            cmap='Blues')
        cax = fig.add_axes([cbarx, cbary, cblen, cbheight])
        fig.colorbar(plot, cax=cax, orientation='horizontal', ticks=(1.e-6,  1.e-2, 1., 1.e2))
        cax.xaxis.set_ticks_position('top')
        cax.set_title(r" $\rho$" + "\n" + "[g cm$^{-3}$]")


        plot = ax[1].pcolormesh(yrange/kpc,zrange/kpc, np.transpose(eint[:,plane,:]),\
                            norm=mcolors.LogNorm(vmin=9.e-21, vmax=5.e-10),
                            cmap='afmhot')
        cax = fig.add_axes([cbarx + 3.*dx1, cbary, cblen, cbheight])
        fig.colorbar(plot, cax=cax, orientation='horizontal', ticks=(1.e-20, 1.e-16, 1.e-10, 1.e-8))
        cax.xaxis.set_ticks_position('top')
        cax.set_title(r" eint" + "\n" + " [cgs]")
        ax[1].tick_params(axis='y', labelleft=False, labelright=False, right=False, left=True)
        ax[1].tick_params(axis='y', labelleft=False, labelright=False, right=False, left=True)


        ax[0].text(0.5, 0.5, '%.2f'%(ds.current_time.to('Myr')))
 
        plt.setp(ax, 'ylim', (-3.9, 4.0))
        plt.title('Lev=%d'%(lev))
        
        outputfile_name =os.path.join(output_folder, 'density-scalar_' + f.split('plt')[1] + '.jpeg')
        plt.savefig(outputfile_name, bbox_inches='tight', dpi=160)

In [ ]:
queue      = Queue()
start_time = ostime.time()
listfile = list_file
num = len(listfile)
infile_list = [infile]*num
num_workers = os.cpu_count()


the_pool = [Process(target=makeSlicePlot, args=(queue,)) for i in range(num_workers)]
for p in the_pool:
    p.start()

for i in range(num):
    queue.put((listfile[i],infile_list[i]))

for i in range(num_workers):
    queue.put(None)

for p in the_pool:
    p.join()

print("Total time= %s seconds ---" % (ostime.time() - start_time))